# Predicting car accident severity using machine learning

This notebook will be used to develop the capstone project of the IBM Certification in Data Science.

The purpose of this project is to predict accident using attributes collected by the Seattle Polive Department and recorded by Traffic Records.


Original dataset: [link_dataset](https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv)
Metadata: [link_metadata](https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Metadata.pdf)



#### Import necessary packages

In [1]:
import numpy as np
import pandas as pd

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
